In [1]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import cv2
import requests
import time

In [2]:
def notifyFile(filename):
    file = {'imageFile':open(filename,'rb')}
    payload = {'message': 'ตรวจพบผู้ไม่สวมหน้ากากอนามัย สถานที่ : ☞ จุดตรวจหน้าอาคาร SC-11 '}
    url = 'https://notify-api.line.me/api/notify'
    token = 'A0IcZtUNMYaWoCPJyc1foPqxxPZik78JNpPAfX8Traa' #EDIT
    headers = {'Authorization':'Bearer ' + token}
    return requests.post(url, headers=headers , data = payload, files=file)

In [3]:
size = (224, 224)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)
model = tensorflow.keras.models.load_model('keras_model.h5')
window_name = 'Detect Mask'
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 1
text_color = (255, 0, 0) 
thickness = 2
cooldown = 20
last_send = time.time() 

In [4]:
cap = cv2.VideoCapture(0)
frame_count = 0
while(True):
    ret, frame = cap.read()
    cv2.imwrite('./image_frame/'+'frame_{}.jpg'.format(frame_count), frame) 
    files = {'src_img':open('./image_frame/'+'frame_{}.jpg'.format(frame_count), 'rb')}
    url = "https://api.aiforthai.in.th/person/human_detect/"
    data = {'json_export':'true','img_export':'true'}
    headers = {
        'Apikey': "E6PjCdHVmnlyoMJx4xCWQpa2DpT4N3BV" #EDIT
    }
    response = requests.post(url, files=files, headers=headers, data=data)
    json = response.json()
    aiforthai = frame.copy()
    persons = json['json_data']
    person_count = 0
    for p in persons:
        print(person_count, p)
        if p['object'] == 'person':
            crop_img = aiforthai[p['y0']:p['y1'], p['x0']:p['x1']]
            cv2.imwrite('person_{}.jpg'.format(person_count), crop_img)
            color = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(color, size, interpolation = cv2.INTER_AREA)
            normalized_image_array = (resized.astype(np.float32) / 127.0) - 1
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            data[0] = normalized_image_array
            prediction = model.predict(data)
            classes = np.argmax(prediction, axis=-1)[0]
            print(classes)
            frame = cv2.rectangle(frame, (p['x0'],p['y0']) , (p['x1'],p['y1']), text_color, thickness) 
            if classes== 0:
                frame = cv2.putText(frame, 'Mask', (p['x0'],p['y0']), font,  
                                    fontScale, text_color, thickness)
            elif classes== 1:
                frame = cv2.putText(frame, 'No Mask', (p['x0'],p['y0']), font, fontScale, text_color, thickness)
                img_name = './image_alert/'+'output_{}.jpg'.format(frame_count)
                cv2.imwrite(img_name, frame)
                if (time.time() - last_send > cooldown):
                    last_send = time.time()
                    notifyFile(img_name)
                    
            else: 
                frame = cv2.putText(frame, '????', (p['x0'],p['y0']), font,  
                                        fontScale, text_color, thickness)
        person_count = person_count + 1
    frame_count = frame_count + 1
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

0 {'object': 'person', 'x0': 100, 'y0': 118, 'x1': 487, 'y1': 417}
1
1 {'object': 'person', 'x0': 98, 'y0': 297, 'x1': 317, 'y1': 414}
0
0 {'object': 'person', 'x0': 92, 'y0': 138, 'x1': 490, 'y1': 424}
1
0 {'object': 'person', 'x0': 148, 'y0': 126, 'x1': 528, 'y1': 418}
1
0 {'object': 'person', 'x0': 141, 'y0': 110, 'x1': 525, 'y1': 418}
1
0 {'object': 'person', 'x0': 173, 'y0': 129, 'x1': 521, 'y1': 423}
0
0 {'object': 'person', 'x0': 164, 'y0': 113, 'x1': 511, 'y1': 419}
0
1 {'object': 'person', 'x0': 94, 'y0': 317, 'x1': 195, 'y1': 410}
0
0 {'object': 'person', 'x0': 156, 'y0': 131, 'x1': 524, 'y1': 416}
0
1 {'object': 'person', 'x0': 91, 'y0': 310, 'x1': 193, 'y1': 411}
0
0 {'object': 'person', 'x0': 156, 'y0': 123, 'x1': 518, 'y1': 421}
0
0 {'object': 'person', 'x0': 157, 'y0': 128, 'x1': 522, 'y1': 420}
0
